# Implement the ansatz

In [67]:
from qiskit.circuit.library.standard_gates import RXGate, RZGate, CXGate, CZGate, SGate, HGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from typing import Dict, Tuple, List

In [2]:
def anzats_circ1(thetas, D2, in_state):
    qr = QuantumRegister(4, name="q")
    qc = QuantumCircuit(qr)
    qc.initialize(in_state)
        
    for d in range(D2):
        qc.append(RXGate(thetas[0]), [qr[0]])
        qc.append(RXGate(thetas[1]), [qr[1]])
        qc.append(RXGate(thetas[2]), [qr[2]])
        qc.append(RXGate(thetas[3]), [qr[3]])
        
        qc.append(RZGate(thetas[4]), [qr[0]])
        qc.append(RZGate(thetas[5]), [qr[1]])
        qc.append(RZGate(thetas[6]), [qr[2]])
        qc.append(RZGate(thetas[7]), [qr[3]])
        
        qc.append(CZGate(), [qr[0], qr[1]])
        qc.append(CZGate(), [qr[1], qr[2]])
        qc.append(CZGate(), [qr[2], qr[3]])
        qc.barrier(qr)
    
    qc.append(RXGate(thetas[0]), [qr[0]])
    qc.append(RXGate(thetas[1]), [qr[1]])
    qc.append(RXGate(thetas[2]), [qr[2]])
    qc.append(RXGate(thetas[3]), [qr[3]])

    qc.append(RZGate(thetas[4]), [qr[0]])
    qc.append(RZGate(thetas[5]), [qr[1]])
    qc.append(RZGate(thetas[6]), [qr[2]])
    qc.append(RZGate(thetas[7]), [qr[3]])
    
    #qc.measure_all()
    
    return qc

def anzats_circ1_uninitialized(thetas, D2):
    qr = QuantumRegister(4, name="q")
    qc = QuantumCircuit(qr)
        
    for d in range(D2):
        qc.append(RXGate(thetas[0]), [qr[0]])
        qc.append(RXGate(thetas[1]), [qr[1]])
        qc.append(RXGate(thetas[2]), [qr[2]])
        qc.append(RXGate(thetas[3]), [qr[3]])
        
        qc.append(RZGate(thetas[4]), [qr[0]])
        qc.append(RZGate(thetas[5]), [qr[1]])
        qc.append(RZGate(thetas[6]), [qr[2]])
        qc.append(RZGate(thetas[7]), [qr[3]])
        
        qc.append(CZGate(), [qr[0], qr[1]])
        qc.append(CZGate(), [qr[1], qr[2]])
        qc.append(CZGate(), [qr[2], qr[3]])
        qc.barrier(qr)
    
    qc.append(RXGate(thetas[0]), [qr[0]])
    qc.append(RXGate(thetas[1]), [qr[1]])
    qc.append(RXGate(thetas[2]), [qr[2]])
    qc.append(RXGate(thetas[3]), [qr[3]])

    qc.append(RZGate(thetas[4]), [qr[0]])
    qc.append(RZGate(thetas[5]), [qr[1]])
    qc.append(RZGate(thetas[6]), [qr[2]])
    qc.append(RZGate(thetas[7]), [qr[3]])
    
    #qc.measure_all()
    
    return qc

def anzats_circ2(phis, D1, in_state):
    qr = QuantumRegister(4, name="q")
    cr = ClassicalRegister(4)
    qc = QuantumCircuit(qr, cr)
    qc.initialize(in_state)
    
    for d in range(D1):
        qc.append(RXGate(phis[0]), [qr[2]])
        qc.append(RXGate(phis[1]), [qr[3]])
        
        qc.append(RZGate(phis[2]), [qr[2]])
        qc.append(RZGate(phis[3]), [qr[3]])
        
        qc.append(CZGate(), [qr[2], qr[3]])
        qc.barrier(qr)
    return qc

# Choose k orthogonal states(computational basis)

In [3]:
import numpy as np
def get_k_basis(k, n):
    full_basis = np.identity(n)
    return full_basis[:k]


# Generating the hamiltonians(for 4 qubits)

In [4]:
from qiskit.opflow import X, Z, I, H

# for convinience let all coefficients ai and Jij be 0.5
H_transverse_ising = 0.5*((I^I^I^X) + (I^I^X^I) + (I^X^I^I) + (X^I^I^I) + \
                          (Z^Z^I^I) + (Z^I^Z^I) + (Z^I^I^Z) + (I^Z^Z^I) + \
                          (I^Z^I^Z) + (I^I^Z^Z))

# TODO: change this to a 4 qubits Hamiltonian
H2_molecule_Hamiltonian = -0.5053051899926562*(I^I) + \
                            -0.3277380754984016*(Z^I) + \
                            0.15567463610622564*(Z^Z) + \
                            -0.3277380754984016*(I^Z)

# Set the parametes of the problam

In [46]:
#subspace size
k = 4

#full space size
QUBITS_NUM = 4 
n = 16 

basis = get_k_basis(k,n)

#number of iterations for each ansatz as discribed in the article
D1 = 1
D2 = 1

#the Hamiltonian
#Ham = H_transverse_ising.to_matrix()
Ham = H_transverse_ising

#Backend number of shots
NUM_SHOTS = 1024

# Expectation Value 

### convert hamiltonian to pauli strings

In [30]:
from qiskit.quantum_info import Pauli

reducing_to_pauli_z_dict = {
    Pauli('I'): Pauli('I'),
    Pauli('Z'): Pauli('Z'),
    Pauli('X'): Pauli('Z'),
    Pauli('Y'): Pauli('Z')
} 

In [91]:
def transfrom_hamiltonian_into_pauli_string(hamiltonian):
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    return pauli_strings

def reduce_pauli_matrixes_into_sigma_z(pauli_string):
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = pauli_string[matrix_index]
        pauli_string[matrix_index].insert(reducing_to_pauli_z_dict[pauli_matrix])
    
    print(pauli_string)
    return pauli_string

def get_z_reduction_for_pauli_string(qc, pauli_string):
    qr = QuantumRegister(4, name="q")
    exdend_qc = QuantumCircuit(qr)
    pauli_string = str(pauli_string)
    for qubit_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            exdend_qc.append(HGate(), [qr[qubit_index]])
        elif pauli_matrix == "Y":
            exdend_qc.append(HGate(), [qr[qubit_index]])
            exdend_qc.append(SGate(), [qr[qubit_index]])
    qc = qc.compose(exdend_qc)
    return qc

### probabilities distribution

In [70]:
def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()
    
    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, simulator_backend) 
    Qobj = assemble(transpiled_quantum_state_circuit)
    result = simulator_backend.run(Qobj).result()
    counts = result.get_counts(quantum_state_circuit)
    
    return get_probability_distribution(counts)

### Expectation value from probabilities

In [71]:
def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, probas_distribution):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

def get_expectation_value(pauli_string, probas_distribution):
        return calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                pauli_string, probas_distribution)
    

# Calculating the first target function

In [89]:
import matplotlib
from qiskit import assemble, Aer
from qiskit import *
from qiskit.visualization import plot_histogram
import math

simulator_backend = Aer.get_backend('aer_simulator') 

def calc_target_func1(thetas, basis, D2, Ham):
    target_func = 0
    pauli_strings = transfrom_hamiltonian_into_pauli_string(Ham)
    
    for j in basis:
        total_expectation_value = 0
        for pauli_string in pauli_strings:
            qc = anzats_circ1(thetas, D2, j)
            qc = get_z_reduction_for_pauli_string(qc, pauli_string)
            probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(qc)
            print(qc.draw())
            total_expectation_value += get_expectation_value(pauli_string, probas_distribution)
        
        target_func += total_expectation_value
        
    return target_func

def objective_func1(thetas):
    target_func = calc_target_func1(thetas, basis, D2, Ham)
    return target_func

# Sending the target function to the BFGS optimazer 

In [92]:
from qiskit.algorithms.optimizers import L_BFGS_B

bfgs_optimizer = L_BFGS_B(maxiter=60)

point, value, nfev = bfgs_optimizer.optimize(8,objective_func1,initial_point=np.zeros(8))
print(point)
print("---point---")
print(value)
print("---value---")

thetas_opt = point


pauli_strings
[Pauli('IIIX'), Pauli('IIXI'), Pauli('IXII'), Pauli('XIII'), Pauli('ZZII'), Pauli('ZIZI'), Pauli('ZIIZ'), Pauli('IZZI'), Pauli('IZIZ'), Pauli('IIZZ')]
        ┌──────────────────────────────────────────────┐┌───────┐┌───────┐   »
   q_0: ┤0                                             ├┤ Rx(0) ├┤ Rz(0) ├─■─»
        │                                              │├───────┤├───────┤ │ »
   q_1: ┤1                                             ├┤ Rx(0) ├┤ Rz(0) ├─■─»
        │  Initialize(1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0) │├───────┤├───────┤   »
   q_2: ┤2                                             ├┤ Rx(0) ├┤ Rz(0) ├───»
        │                                              │├───────┤├───────┤   »
   q_3: ┤3                                             ├┤ Rx(0) ├┤ Rz(0) ├───»
        └──────────────────────────────────────────────┘└───────┘└───────┘   »
meas: 4/═════════════════════════════════════════════════════════════════════»
                                             

        ┌──────────────────────────────────────────────┐┌───────┐┌───────┐   »
   q_0: ┤0                                             ├┤ Rx(0) ├┤ Rz(0) ├─■─»
        │                                              │├───────┤├───────┤ │ »
   q_1: ┤1                                             ├┤ Rx(0) ├┤ Rz(0) ├─■─»
        │  Initialize(1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0) │├───────┤├───────┤   »
   q_2: ┤2                                             ├┤ Rx(0) ├┤ Rz(0) ├───»
        │                                              │├───────┤├───────┤   »
   q_3: ┤3                                             ├┤ Rx(0) ├┤ Rz(0) ├───»
        └──────────────────────────────────────────────┘└───────┘└───────┘   »
meas: 4/═════════════════════════════════════════════════════════════════════»
                                                                             »
«               ░ ┌───────┐┌───────┐ ░ ┌─┐         
«   q_0: ───────░─┤ Rx(0) ├┤ Rz(0) ├─░─┤M├─────────
«               ░ ├───────┤

        ┌──────────────────────────────────────────────┐┌───────┐┌───────┐   »
   q_0: ┤0                                             ├┤ Rx(0) ├┤ Rz(0) ├─■─»
        │                                              │├───────┤├───────┤ │ »
   q_1: ┤1                                             ├┤ Rx(0) ├┤ Rz(0) ├─■─»
        │  Initialize(0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0) │├───────┤├───────┤   »
   q_2: ┤2                                             ├┤ Rx(0) ├┤ Rz(0) ├───»
        │                                              │├───────┤├───────┤   »
   q_3: ┤3                                             ├┤ Rx(0) ├┤ Rz(0) ├───»
        └──────────────────────────────────────────────┘└───────┘└───────┘   »
meas: 4/═════════════════════════════════════════════════════════════════════»
                                                                             »
«               ░ ┌───────┐┌───────┐ ░ ┌─┐         
«   q_0: ───────░─┤ Rx(0) ├┤ Rz(0) ├─░─┤M├─────────
«               ░ ├───────┤

        ┌──────────────────────────────────────────────┐┌───────┐┌───────┐   »
   q_0: ┤0                                             ├┤ Rx(0) ├┤ Rz(0) ├─■─»
        │                                              │├───────┤├───────┤ │ »
   q_1: ┤1                                             ├┤ Rx(0) ├┤ Rz(0) ├─■─»
        │  Initialize(0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0) │├───────┤├───────┤   »
   q_2: ┤2                                             ├┤ Rx(0) ├┤ Rz(0) ├───»
        │                                              │├───────┤├───────┤   »
   q_3: ┤3                                             ├┤ Rx(0) ├┤ Rz(0) ├───»
        └──────────────────────────────────────────────┘└───────┘└───────┘   »
meas: 4/═════════════════════════════════════════════════════════════════════»
                                                                             »
«               ░ ┌───────┐┌───────┐ ░ ┌─┐         
«   q_0: ───────░─┤ Rx(0) ├┤ Rz(0) ├─░─┤M├─────────
«               ░ ├───────┤

        ┌──────────────────────────────────────────────┐┌───────┐┌───────┐   »
   q_0: ┤0                                             ├┤ Rx(0) ├┤ Rz(0) ├─■─»
        │                                              │├───────┤├───────┤ │ »
   q_1: ┤1                                             ├┤ Rx(0) ├┤ Rz(0) ├─■─»
        │  Initialize(0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0) │├───────┤├───────┤   »
   q_2: ┤2                                             ├┤ Rx(0) ├┤ Rz(0) ├───»
        │                                              │├───────┤├───────┤   »
   q_3: ┤3                                             ├┤ Rx(0) ├┤ Rz(0) ├───»
        └──────────────────────────────────────────────┘└───────┘└───────┘   »
meas: 4/═════════════════════════════════════════════════════════════════════»
                                                                             »
«               ░ ┌───────┐┌───────┐ ░ ┌─┐         
«   q_0: ───────░─┤ Rx(0) ├┤ Rz(0) ├─░─┤M├─────────
«               ░ ├───────┤

...

        ┌──────────────────────────────────────────────┐┌───────┐  ┌───────┐  »
   q_0: ┤0                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │                                              │├───────┤  ├───────┤  »
   q_1: ┤1                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │  Initialize(1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0) │├───────┤  ├───────┤  »
   q_2: ┤2                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │                                              │├───────┤┌─┴───────┴─┐»
   q_3: ┤3                                             ├┤ Rx(0) ├┤ Rz(1e-08) ├»
        └──────────────────────────────────────────────┘└───────┘└───────────┘»
meas: 4/══════════════════════════════════════════════════════════════════════»
                                                                              »
«                  ░ ┌───────┐  ┌───────┐   ░ ┌─┐         
«   q_0: ─■────────░─┤ Rx(0) ├──┤ Rz(0) ├───░─┤M├─────────
« 

        ┌──────────────────────────────────────────────┐┌───────┐  ┌───────┐  »
   q_0: ┤0                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │                                              │├───────┤  ├───────┤  »
   q_1: ┤1                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │  Initialize(0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0) │├───────┤  ├───────┤  »
   q_2: ┤2                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │                                              │├───────┤┌─┴───────┴─┐»
   q_3: ┤3                                             ├┤ Rx(0) ├┤ Rz(1e-08) ├»
        └──────────────────────────────────────────────┘└───────┘└───────────┘»
meas: 4/══════════════════════════════════════════════════════════════════════»
                                                                              »
«                  ░ ┌───────┐  ┌───────┐  ┌───┐ ░ ┌─┐         
«   q_0: ─■────────░─┤ Rx(0) ├──┤ Rz(0) ├──┤ H ├─░─┤M├──

        ┌──────────────────────────────────────────────┐┌───────┐  ┌───────┐  »
   q_0: ┤0                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │                                              │├───────┤  ├───────┤  »
   q_1: ┤1                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │  Initialize(0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0) │├───────┤  ├───────┤  »
   q_2: ┤2                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │                                              │├───────┤┌─┴───────┴─┐»
   q_3: ┤3                                             ├┤ Rx(0) ├┤ Rz(1e-08) ├»
        └──────────────────────────────────────────────┘└───────┘└───────────┘»
meas: 4/══════════════════════════════════════════════════════════════════════»
                                                                              »
«                  ░ ┌───────┐  ┌───────┐        ░ ┌─┐         
«   q_0: ─■────────░─┤ Rx(0) ├──┤ Rz(0) ├────────░─┤M├──

        ┌──────────────────────────────────────────────┐┌───────┐  ┌───────┐  »
   q_0: ┤0                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │                                              │├───────┤  ├───────┤  »
   q_1: ┤1                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │  Initialize(0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0) │├───────┤  ├───────┤  »
   q_2: ┤2                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │                                              │├───────┤┌─┴───────┴─┐»
   q_3: ┤3                                             ├┤ Rx(0) ├┤ Rz(1e-08) ├»
        └──────────────────────────────────────────────┘└───────┘└───────────┘»
meas: 4/══════════════════════════════════════════════════════════════════════»
                                                                              »
«                  ░ ┌───────┐  ┌───────┐   ░ ┌─┐         
«   q_0: ─■────────░─┤ Rx(0) ├──┤ Rz(0) ├───░─┤M├─────────
« 

        ┌──────────────────────────────────────────────┐┌───────┐  ┌───────┐  »
   q_0: ┤0                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │                                              │├───────┤  ├───────┤  »
   q_1: ┤1                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │  Initialize(0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0) │├───────┤  ├───────┤  »
   q_2: ┤2                                             ├┤ Rx(0) ├──┤ Rz(0) ├──»
        │                                              │├───────┤┌─┴───────┴─┐»
   q_3: ┤3                                             ├┤ Rx(0) ├┤ Rz(1e-08) ├»
        └──────────────────────────────────────────────┘└───────┘└───────────┘»
meas: 4/══════════════════════════════════════════════════════════════════════»
                                                                              »
«                  ░ ┌───────┐  ┌───────┐   ░ ┌─┐         
«   q_0: ─■────────░─┤ Rx(0) ├──┤ Rz(0) ├───░─┤M├─────────
« 

# Calculating the second target function

In [ ]:
def calc_target_func2(thetas_opt, phis, in_state, D1, D2, Ham):
    target_func = 0

    qc2 = anzats_circ2(phis, D1, in_state)
    qc1 = anzats_circ1_uninitialized(thetas_opt, D2)
    qc = qc2.compose(qc1)
    print(qc.draw())
    qobj = assemble(qc)
    result = sim.run(qobj).result().get_counts()
    state = convert_result_to_state(result)
    dagger_state = np.matrix(state)
    state = dagger_state.getH()
    state = np.array(state)
    dagger_state = np.array(dagger_state)
        
#         TODO: calculate this with estimation with the Ham pauli strings
#         if Ham == "Ising Model":
#             product = calc_ising_avg(qc)
#         else             product = calc_molecular_avg(qc)
    product = np.matmul(Ham, state)
    product = np.matmul(dagger_state, product)

    target_func += product
        
    return target_func

i = np.random.randint(0,k)
def objective_func2(phis):
    in_state = basis[i]
    target_func2 = calc_target_func2(thetas_opt, phis, in_state, D1, D2, Ham)
    print("target func:")
    print(target_func2[0][0])
    return target_func2[0][0]

def objective_func2_neg(phis):
    return -1*objective_func2(phis)

# Sending the target function to the BFGS optimazer 

In [ ]:
point, value, nfev = bfgs_optimizer.optimize(4, objective_func2_neg, initial_point=np.array([1, 1, 1, 1]))
print(point)
print("---point---")
print(value)
print("---value---")

The minimal value of ⟨ψ|H|ψ⟩ is:

In [ ]:
value = -value
print(value)